In [1]:
# Imports
import pandas as pd
import sys
import glob
import errno
import csv
import numpy as np
from nltk.corpus import stopwords
import re
import nltk.data
import nltk
import os
from collections import OrderedDict
from subprocess import check_call
from shutil import copyfile
from sklearn.metrics import log_loss
%matplotlib inline
import matplotlib.pyplot as plt
import mpld3
mpld3.enable_notebook()
import seaborn as sns
from collections import Counter
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from tqdm import tqdm
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization

/Users/16521/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [208]:
# Read data
# train = "../data/train.csv"
# test = "../data/test.csv"
wv = "../../../../glove.6B/glove.6B.100d.txt"
X_train = pd.read_csv( train, header=0,delimiter="," )
X_test = pd.read_csv( test, header=0,delimiter="," )

word_vecs = {}
with open(wv) as f:
    for line in f:
       vals = line.split()
       word_vecs[vals[0]] = np.array(vals[1::],dtype=float)
authors = ['EAP','MWS','HPL']

Y_train = LabelEncoder().fit_transform(X_train['author'])

In [3]:
# Clean data
def getWordVectors(X_train,X_test,word_vecs):
    X_train['word_vectors'] = [ [ word_vecs[word] for word in sentence if word in word_vecs] for sentence in X_train['text']]
    X_test['word_vectors'] = [ [ word_vecs[word] for word in sentence if word in word_vecs] for sentence in X_test['text']] 
    return X_train,X_test

def getSentenceVectors(X_train,X_test):
    X_train['sentence_vectors'] =[np.mean(sentence,axis = 0) for sentence in X_train['word_vectors']]
    X_test['sentence_vectors'] =[np.mean(sentence,axis = 0) for sentence in X_test['word_vectors']] 
    return X_train,X_test

def clean(X_train,X_test):
    X_train['words'] = [re.sub("[^a-zA-Z]"," ", data).lower().split() for data in X_train['text']]
    X_test['words'] = [re.sub("[^a-zA-Z]"," ", data).lower().split() for data in X_test['text']]
    return X_train,X_test
X_train,X_test = clean(X_train,X_test)
X_train,X_test = getWordVectors(X_train,X_test,word_vecs)
X_train,X_test = getSentenceVectors(X_train,X_test)
#X_train.head()

In [37]:
# Feature Engineering
# Punctuation
punctuations = [{"id":1,"p":"[;:]"},{"id":2,"p":"[,.]"},{"id":3,"p":"[?]"},{"id":4,"p":"[\']"},{"id":5,"p":"[\"]"},{"id":6,"p":"[;:,.?\'\"]"}]
for p in punctuations:
    punctuation = p["p"]
    _train =  [ sentence.split() for sentence in X_train['text'] ]
    X_train['punc_'+str(p["id"])] = [len([word for word in sentence if bool(re.search(punctuation, word))])*100.0/len(sentence) for sentence in _train]    

    _test =  [ sentence.split() for sentence in X_test['text'] ]
    X_test['punc_'+str(p["id"])] = [len([word for word in sentence if bool(re.search(punctuation, word))])*100.0/len(sentence) for sentence in _test]    



In [38]:
# Feature Engineering
# Stop Words
_dist_train = [x for x in X_train['words']]
X_train['stop_word'] = [len([word for word in sentence if word in stopwords.words('english')])*100.0/len(sentence) for sentence in _dist_train]

_dist_test = [x for x in X_test['words']]
X_test['stop_word'] = [len([word for word in sentence if word in stopwords.words('english')])*100.0/len(sentence) for sentence in _dist_test]    

In [39]:
# Feature Engineering
# tfidf - words - nb+svd
def tfidfWords(X_train,X_test):
    tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
    full_tfidf = tfidf_vec.fit_transform(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train['text'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['text'].values.tolist())
    return train_tfidf,test_tfidf,full_tfidf
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def runSVD(full_tfidf,train_tfidf,test_tfidf):   
    n_comp = 20
    svd_obj = TruncatedSVD(n_components=n_comp)
    svd_obj.fit(full_tfidf)
    train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
    test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))

    train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
    test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
    return train_svd,test_svd

def do_tfidf_MNB(X_train,X_test,Y_train):
    train_tfidf,test_tfidf,full_tfidf = tfidfWords(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 3])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

def do_tfidf_SVD(X_train,X_test,Y_train):
    train_tfidf,test_tfidf,full_tfidf = tfidfWords(X_train,X_test)
    train_svd,test_svd = runSVD(full_tfidf,train_tfidf,test_tfidf)
    return train_svd,test_svd

pred_train,pred_test = do_tfidf_MNB(X_train,X_test,Y_train)
X_train["tfidf_words_nb_eap"] = pred_train[:,0]
X_train["tfidf_words_nb_hpl"] = pred_train[:,1]
X_train["tfidf_words_nb_mws"] = pred_train[:,2]
X_test["tfidf_words_nb_eap"] = pred_test[:,0]
X_test["tfidf_words_nb_hpl"] = pred_test[:,1]
X_test["tfidf_words_nb_mws"] = pred_test[:,2]

# pred_train,pred_test = do_tfidf_SVD(X_train,X_test,Y_train)
# print pred_train
# # X_train["tfidf_words_nb_eap"] = pred_train[:,0]
# # X_train["tfidf_words_nb_hpl"] = pred_train[:,1]
# # X_train["tfidf_words_nb_mws"] = pred_train[:,2]
# # X_test["tfidf_words_nb_eap"] = pred_test[:,0]
# # X_test["tfidf_words_nb_hpl"] = pred_test[:,1]
# # X_test["tfidf_words_nb_mws"] = pred_test[:,2]

('Mean cv score : ', 0.84221619836128525)


In [40]:
# Feature Engineering
# tfidf - chars - nb+svd
def tfidfWords(X_train,X_test):
    tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,5),analyzer='char')
    full_tfidf = tfidf_vec.fit_transform(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train['text'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['text'].values.tolist())
    return train_tfidf,test_tfidf
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do(X_train,X_test,Y_train):
    train_tfidf,test_tfidf = tfidfWords(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 3])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test
pred_train,pred_test = do(X_train,X_test,Y_train)
X_train["tfidf_chars_nb_eap"] = pred_train[:,0]
X_train["tfidf_chars_nb_hpl"] = pred_train[:,1]
X_train["tfidf_chars_nb_mws"] = pred_train[:,2]
X_test["tfidf_chars_nb_eap"] = pred_test[:,0]
X_test["tfidf_chars_nb_hpl"] = pred_test[:,1]
X_test["tfidf_chars_nb_mws"] = pred_test[:,2]

('Mean cv score : ', 0.7904152589474216)


In [41]:
# Feature Engineering
# count - words - nb
def countWords(X_train,X_test):
    count_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
    count_vec.fit(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_count = count_vec.transform(X_train['text'].values.tolist())
    test_count = count_vec.transform(X_test['text'].values.tolist())
    return train_count,test_count
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do_count_MNB(X_train,X_test,Y_train):
    train_count,test_count=countWords(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 3])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_count[dev_index], train_count[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_count)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

pred_train,pred_test = do_count_MNB(X_train,X_test,Y_train)
X_train["count_words_nb_eap"] = pred_train[:,0]
X_train["count_words_nb_hpl"] = pred_train[:,1]
X_train["count_words_nb_mws"] = pred_train[:,2]
X_test["count_words_nb_eap"] = pred_test[:,0]
X_test["count_words_nb_hpl"] = pred_test[:,1]
X_test["count_words_nb_mws"] = pred_test[:,2]

('Mean cv score : ', 0.45091841616567468)


In [42]:
# Feature Engineering
# count - chars - nb
def countChars(X_train,X_test):
    count_vec = CountVectorizer(ngram_range=(1,7),analyzer='char')
    count_vec.fit(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_count = count_vec.transform(X_train['text'].values.tolist())
    test_count = count_vec.transform(X_test['text'].values.tolist())
    return train_count,test_count
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do_count_chars_MNB(X_train,X_test,Y_train):
    train_count,test_count=countChars(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 3])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_count[dev_index], train_count[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_count)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

pred_train,pred_test = do_count_chars_MNB(X_train,X_test,Y_train)
X_train["count_chars_nb_eap"] = pred_train[:,0]
X_train["count_chars_nb_hpl"] = pred_train[:,1]
X_train["count_chars_nb_mws"] = pred_train[:,2]
X_test["count_chars_nb_eap"] = pred_test[:,0]
X_test["count_chars_nb_hpl"] = pred_test[:,1]
X_test["count_chars_nb_mws"] = pred_test[:,2]

('Mean cv score : ', 3.750763922681903)


In [105]:
# Word Embeddings
# this function creates a normalized vector for the whole sentence
#X_train['sentence_vectors'][0]

#foo.columns = ['svd_char_'+str(i) for i in range(n_comp)]
X_train = pd.concat([X_train,pd.DataFrame(X_train['sentence_vectors'].tolist())],axis=1)
X_test = pd.concat([X_test,pd.DataFrame(X_test['sentence_vectors'].tolist())],axis=1)

# xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
# xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]
# xtrain_glove = np.array(xtrain_glove)
# xvalid_glove = np.array(xvalid_glove)

In [213]:
# load the GloVe vectors in a dictionary:

def loadWordVecs():
    embeddings_index = {}
    f = open(wv)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    return embeddings_index

def sent2vec(embeddings_index,s): # this function creates a normalized vector for the whole sentence
    words = str(s).lower().decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stopwords.words('english')]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(100)
    return v / np.sqrt((v ** 2).sum())

def doGlove(x_train,x_test):
    embeddings_index = loadWordVecs()
    # create sentence vectors using the above function for training and validation set
    xtrain_glove = [sent2vec(embeddings_index,x) for x in tqdm(x_train)]
    xtest_glove = [sent2vec(embeddings_index,x) for x in tqdm(x_test)]
    xtrain_glove = np.array(xtrain_glove)
    xtest_glove = np.array(xtest_glove)
    return xtrain_glove,xtest_glove,embeddings_index

glove_vecs_train,glove_vecs_test,embeddings_index = doGlove(X_train['text'],X_test['text'])
X_train[['sent_vec_'+str(i) for i in range(100)]] = pd.DataFrame(glove_vecs_train.tolist())
X_test[['sent_vec_'+str(i) for i in range(100)]] = pd.DataFrame(glove_vecs_test.tolist())



Found 400000 word vectors.


In [215]:
# CNN

def doAddNN(X_train,X_test,pred_train,pred_test):
    X_train["nn_eap"] = pred_train[:,0]
    X_train["nn_hpl"] = pred_train[:,1]
    X_train["nn_mws"] = pred_train[:,2]
    X_test["nn_eap"] = pred_test[:,0]
    X_test["nn_hpl"] = pred_test[:,1]
    X_test["nn_mws"] = pred_test[:,2]
    return X_train,X_test

def initNN():
    # create a simple 3 layer sequential neural net
    model = Sequential()

    model.add(Dense(40, input_dim=50, activation='relu'))
    model.add(Dropout(0.6))
    model.add(BatchNormalization())

    model.add(Dense(40, activation='relu'))
    model.add(Dropout(0.6))
    model.add(BatchNormalization())

    model.add(Dense(3))
    model.add(Activation('softmax'))

    # compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def initCNN(word_index,embedding_matrix,max_len):
    # A simple CNN with glove embeddings and two dense layers
    model = Sequential([
        Embedding(len(word_index) + 1,30,input_length=max_len),
        Dropout(0.5),
        Conv1D(40, 3, border_mode='same', activation='relu'),
        Dropout(0.5),
        MaxPooling1D(),
        Flatten(),
        Dense(40, activation='relu'),
        Dropout(0.6),
        Dense(3, activation='softmax')])
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def doNN(X_train,X_test):
    #glove_vecs_train,glove_vecs_test = doGlove(X_train['text'],X_test['text'])
    # scale the data before any neural net:
    token = text.Tokenizer(num_words=None)
    max_len = 70

    token.fit_on_texts(list(X_train['text']) + list(X_test['text']))
    xtrain_seq = token.texts_to_sequences(X_train['text'])
    xtest_seq = token.texts_to_sequences(X_test['text'])

    # zero pad the sequences
    xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
    xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

    word_index = token.word_index

    # create an embedding matrix for the words we have in the dataset
    embedding_matrix = np.zeros((len(word_index) + 1, 100))
    for word, i in tqdm(word_index.items()):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    # we need to binarize the labels for the neural net
    ytrain_enc = np_utils.to_categorical(Y_train)
    #yvalid_enc = np_utils.to_categorical(yvalid)



    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([xtrain_pad.shape[0], 3])
    for dev_index, val_index in kf.split(xtrain_pad):
        dev_X, val_X = xtrain_pad[dev_index], xtrain_pad[val_index]
        dev_y, val_y = ytrain_enc[dev_index], ytrain_enc[val_index]
        model = initCNN(word_index,embedding_matrix,max_len)
        model.fit(dev_X, y=dev_y, batch_size=64, epochs=5, verbose=1,validation_data=(val_X, val_y))
        pred_val_y = model.predict(val_X)
        pred_test_y = model.predict(xtest_pad)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
    return doAddNN(X_train,X_test,pred_train,pred_full_test/5)
X_train,X_test = doNN(X_train,X_test)

100%|██████████| 29451/29451 [00:00<00:00, 329204.58it/s]
/Users/16521/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(40, 3, padding="same", activation="relu")`


Train on 15663 samples, validate on 3916 samples
Epoch 1/5
15663/15663 [==============================] - 15s 944us/step - loss: 1.0777 - val_loss: 1.0545
Epoch 2/5
15663/15663 [==============================] - 5s 339us/step - loss: 0.8259 - val_loss: 0.5658
Epoch 3/5
15663/15663 [==============================] - 5s 345us/step - loss: 0.4621 - val_loss: 0.4317
Epoch 4/5
15663/15663 [==============================] - 5s 344us/step - loss: 0.3087 - val_loss: 0.4116
Epoch 5/5
15663/15663 [==============================] - 5s 348us/step - loss: 0.2276 - val_loss: 0.4149
Train on 15663 samples, validate on 3916 samples
Epoch 1/5
15663/15663 [==============================] - 15s 952us/step - loss: 1.0763 - val_loss: 1.0227
Epoch 2/5
15663/15663 [==============================] - 5s 340us/step - loss: 0.7522 - val_loss: 0.5439
Epoch 3/5
15663/15663 [==============================] - 6s 353us/step - loss: 0.4208 - val_loss: 0.4500
Epoch 4/5
15663/15663 [==============================] - 5s 

(0.048079044, 0.94079792, 0.011123078)

In [ ]:
# Model
# XGBoost
def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.3):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 3
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = child
    param['subsample'] = 0.8
    param['colsample_bytree'] = colsample
    param['seed'] = seed_val
    num_rounds = 2000

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest, ntree_limit = model.best_ntree_limit)
    if test_X2 is not None:
        xgtest2 = xgb.DMatrix(test_X2)
        pred_test_y2 = model.predict(xgtest2, ntree_limit = model.best_ntree_limit)
    return pred_test_y, pred_test_y2, model

def do(X_train,X_test,Y_train):
    drop_columns=["id","text","words","word_vectors","sentence_vectors"]
    x_train = X_train.drop(drop_columns+['author'],axis=1)
    x_test = X_test.drop(drop_columns,axis=1)
    y_train = Y_train
    
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([x_train.shape[0], 3])
    for dev_index, val_index in kf.split(x_train):
        dev_X, val_X = x_train.loc[dev_index], x_train.loc[val_index]
        dev_y, val_y = y_train[dev_index], y_train[val_index]
        pred_val_y, pred_test_y, model = runXGB(dev_X, dev_y, val_X, val_y, x_test, seed_val=0, colsample=0.7)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("cv scores : ", cv_scores)
    return pred_full_test/5
result = do(X_train,X_test,Y_train)

[0]	train-mlogloss:0.999628	test-mlogloss:0.999482
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
[20]	train-mlogloss:0.391298	test-mlogloss:0.397139
[40]	train-mlogloss:0.319198	test-mlogloss:0.333202
[60]	train-mlogloss:0.296807	test-mlogloss:0.320042
[80]	train-mlogloss:0.281649	test-mlogloss:0.314684
[100]	train-mlogloss:0.26805	test-mlogloss:0.311341
[120]	train-mlogloss:0.257358	test-mlogloss:0.309619
[140]	train-mlogloss:0.247042	test-mlogloss:0.308337
[160]	train-mlogloss:0.237471	test-mlogloss:0.307778
[180]	train-mlogloss:0.229021	test-mlogloss:0.306946
[200]	train-mlogloss:0.220467	test-mlogloss:0.306314
[220]	train-mlogloss:0.21224	test-mlogloss:0.306209
[240]	train-mlogloss:0.2049	test-mlogloss:0.305659
[260]	train-mlogloss:0.198042	test-mlogloss:0.306052
[280]	train-mlogloss:0.191251	test-mlogloss:0.306039
[300]	train-mlogloss:0.185274	test-mlogloss:0.306677
Stopping. B

In [189]:
# Write Results

def writeResult(result,test):
    # count number of files
    path, dirs, files = os.walk("../results").next()
    file_count = len(files)/2+1

    # Write the test results
    data=OrderedDict()
    data["id"]=test["id"] 
    data["EAP"]=result[0]#["EAP"]
    data["HPL"]=result[1]#["HPL"]	
    data["MWS"]=result[2]#["MWS"]
    output = pd.DataFrame(data=data)
    filename = "../results/result"+str(file_count)+".csv"
    output.to_csv( filename, index=False )
    filename = "../results/result"+str(file_count)+"compr.csv"
    output.to_csv( filename, index=False )
    check_call(['gzip', filename])

writeResult(result.T,X_test)

Index([u'id', u'text', u'author', u'words', u'word_vectors',
       u'sentence_vectors', u'punc_1', u'punc_2', u'punc_3', u'punc_4',
       ...
       u'sent_vec_290', u'sent_vec_291', u'sent_vec_292', u'sent_vec_293',
       u'sent_vec_294', u'sent_vec_295', u'sent_vec_296', u'sent_vec_297',
       u'sent_vec_298', u'sent_vec_299'],
      dtype='object', length=328)

Index([u'id', u'text', u'author', u'words', u'word_vectors',
       u'sentence_vectors', u'sent_vec_0', u'sent_vec_1', u'sent_vec_2',
       u'sent_vec_3', u'sent_vec_4', u'sent_vec_5', u'sent_vec_6',
       u'sent_vec_7', u'sent_vec_8', u'sent_vec_9', u'sent_vec_10',
       u'sent_vec_11', u'sent_vec_12', u'sent_vec_13', u'sent_vec_14',
       u'sent_vec_15', u'sent_vec_16', u'sent_vec_17', u'sent_vec_18',
       u'sent_vec_19', u'sent_vec_20', u'sent_vec_21', u'sent_vec_22',
       u'sent_vec_23', u'sent_vec_24', u'sent_vec_25', u'sent_vec_26',
       u'sent_vec_27', u'sent_vec_28', u'sent_vec_29', u'sent_vec_30',
       u'sent_vec_31', u'sent_vec_32', u'sent_vec_33', u'sent_vec_34',
       u'sent_vec_35', u'sent_vec_36', u'sent_vec_37', u'sent_vec_38',
       u'sent_vec_39', u'sent_vec_40', u'sent_vec_41', u'sent_vec_42',
       u'sent_vec_43', u'sent_vec_44', u'sent_vec_45', u'sent_vec_46',
       u'sent_vec_47', u'sent_vec_48', u'sent_vec_49', u'nn_eap', u'nn_hpl',
       u'nn_mws

In [149]:
X_train.head()

,id,text,author,words,word_vectors,sentence_vectors,nn_eap,nn_hpl,nn_mws,punc_1,...,sent_vec_90,sent_vec_91,sent_vec_92,sent_vec_93,sent_vec_94,sent_vec_95,sent_vec_96,sent_vec_97,sent_vec_98,sent_vec_99
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, however, afforded, me, no, mea...","[[-0.22701, 0.70329, 0.96125, 0.93479, 0.7205,...","[-0.102682611702, 0.843608723404, 0.6941906914...",0.996464,0.002704,0.000832,4.878049,...,-0.064199,-0.051707,-0.032212,-0.015696,-0.039146,-0.034641,-0.026215,-0.079093,0.068413,0.062398
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, once, occurred, to, me, that, the,...","[[-0.37915, 0.61848, 0.9593, 0.90403, 0.36806,...","[-0.071579877193, 0.818990877193, 0.7487534035...",0.747714,0.200467,0.051819,0.000000,...,-0.042077,0.016603,-0.056828,0.019743,-0.083541,0.018332,-0.020753,-0.027814,0.011478,0.054295
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, his, left, hand, was, a, gold, snuff, box...","[[-0.27004, 1.1144, 1.0493, 0.57924, 0.78968, ...","[-0.137265402439, 0.810834207317, 0.6727634268...",0.849465,0.114169,0.036367,0.000000,...,-0.065868,-0.024816,-0.029570,0.050657,-0.152419,-0.024303,-0.033989,-0.062024,0.138996,0.046701
3,id27763,How lovely is spring As we looked from Windsor...,MWS,"[how, lovely, is, spring, as, we, looked, from...","[[-0.043861, 1.3183, -0.03715, 0.85478, 0.1221...","[-0.0491552662722, 0.823676804734, 0.681447236...",0.002261,0.003433,0.994305,0.000000,...,0.014403,-0.109733,0.002690,0.046515,-0.238110,0.006557,-0.143425,0.060398,0.095089,0.036426
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,"[finding, nothing, else, not, even, gold, the,...","[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...","[-0.0283006027397, 0.804272054795, 0.685995212...",0.104045,0.710142,0.185813,3.703704,...,0.049627,-0.031102,0.043574,-0.036604,-0.119386,-0.016236,-0.116524,-0.016949,0.043204,0.084453


In [207]:
X_train = X_train.drop(['sent_vec_'+str(i) for i in range(300)],axis=1)

In [182]:
result[0]

array([ 0.00833853,  0.0029562 ,  1.48870525])